<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/Slicing_models_feature_Soft_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install torch
!pip install tqdm
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manyli

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [13]:
# 모델 로드
model_paths = {
    "codebert": "wwookk/slice_code_feature",
    "graphcodebert": "wwookk/slice_graph_feature",
    "unixcoder": "wwookk/slice_unix_feature"
}

models = {name: RobertaForSequenceClassification.from_pretrained(path) for name, path in model_paths.items()}

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base-mlm")

You are using a model of type codebert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wwookk/slice_code_feature and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a model of type codebert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wwookk/slice_graph_feature and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-str

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
# 코드 전처리 함수
def preprocess_code(code_text):
    code_text = re.sub(r'/\*.*?\*/', "", code_text, flags=re.DOTALL)
    code_text = re.sub(r'//.*?\n', "", code_text)
    code_text = re.sub(r'#include .*?\n', "", code_text)
    code_text = re.sub(r'#include "*.*?\n', "", code_text)
    code_text = re.sub(r'#define .*?\n', "", code_text)
    code_text = re.sub(r'#undef\s+\w+', "", code_text)
    code_text = re.sub(r'#if\s+\w+', "", code_text)
    code_text = re.sub(r'#else\s+\w+', "", code_text)
    code_text = re.sub(r'#elif\s+\w+', "", code_text)
    code_text = re.sub(r'#endif', "", code_text)
    code_text = re.sub(r'[\t ]+', " ", code_text)
    code_text = re.sub(r'\n\s*\n', "\n", code_text)
    code_text = re.sub(r'\n', " ", code_text)
    code_text = re.sub(r'return.*?;', "", code_text)
    code_text = re.sub(r'return;', "", code_text)
    return code_text

In [15]:
# 예측 함수
def predict_class(input_code, models, tokenizer):
    # 입력 코드 전처리
    preprocessed_code = preprocess_code(input_code)

    # 입력 코드 토크나이징
    inputs = tokenizer(preprocessed_code, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 모든 모델의 예측을 소프트맥스 확률로 합산
    sum_probs = None
    for model in models.values():
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=-1)
            sum_probs = probs if sum_probs is None else sum_probs + probs

    # 평균 확률 계산
    avg_probs = sum_probs / len(models)

    # 가장 높은 확률을 가진 클래스 선택
    predicted_class = torch.argmax(avg_probs, dim=-1).item()
    return predicted_class

In [16]:
# CWE ID 매핑
predicted_class_label = {
    0: "CWE-415 취약점\nDouble Free",
    1: "CWE-119 취약점\nImproper Restriction of Operations within the Bounds of a Memory Buffer",
    2: "CWE-20 취약점\nImproper Input Validation",
    3: "CWE-125 취약점\nOut-of-bounds Read",
    4: "CWE-787 취약점\nOut-of-bounds Write",
    5: "CWE-416 취약점\nUse after Free",
    6: "CWE-476 취약점\nNULL Pointer Dereference",
    7: "CWE-399 취약점\nResource Management Errors",
    8: "CWE-190 취약점\nInteger Overflow or Wraparound"
}

In [19]:
# 입력 코드 예시
input_code = """
char* ptr = (char*)malloc (SIZE);
...
if (abrt) {
free(ptr);
}
...
free(ptr);
"""

# 클래스 예측 실행
predicted_label = predict_class(input_code, models, tokenizer)
predicted_class_label_text = predicted_class_label[predicted_label]
print(f"The predicted CWE ID is: {predicted_class_label_text}")

The predicted CWE ID is: CWE-119 취약점
Improper Restriction of Operations within the Bounds of a Memory Buffer
